In [1]:
%matplotlib inline



# Write user defined Operators as a package
This example shows how more complex DPF python plugins of Operators can be created as standard python packages.
The benefits of writing packages instead of simple scripts are: componentization (split the code in several
python modules or files), distribution (with packages, standard python tools can be used to upload and
download packages) and documentation (READMEs, docs, tests and examples can be added to the package).

This plugin will hold 2 different Operators:
  - One returning all the scoping ids having data higher than the average
  - One returning all the scoping ids having data lower than the average


## Write Operator
For this more advanced use case, a python package is created.
Each Operator implementation derives from `ansys.dpf.core.custom_operator.CustomOperatorBase`
and a call to `ansys.dpf.core.custom_operator.record_operator` records the Operators of the plugin.
The complete package looks like:



In [3]:
import IPython
import os
print('\033[1m average_filter_plugin')
print('\033[1m __init__.py:\n \033[0m')
print(IPython.display.Code(os.path.join(os.getcwd(),"../plugins/average_filter_plugin/__init__.py")))
print('\n\n\033[1m operators.py:\n \033[0m')
print(IPython.display.Code(os.path.join(os.getcwd(),"../plugins/average_filter_plugin/operators.py")))
print('\n\n\033[1m operators_loader.py:\n \033[0m')
print(IPython.display.Code(os.path.join(os.getcwd(),"../plugins/average_filter_plugin/operators_loader.py")))
print('\n\n\033[1m common.py:\n \033[0m')
print(IPython.display.Code(os.path.join(os.getcwd(),"../plugins/average_filter_plugin/common.py")))


 average_filter_plugin
 __init__.py:
 
from average_filter_plugin.operators_loader import load_operators



 operators.py:
 
from ansys.dpf.core.custom_operator import CustomOperatorBase
from ansys.dpf.core.operator_specification import CustomSpecification, PinSpecification, SpecificationProperties
from ansys.dpf import core as dpf
from average_filter_plugin import common


class IdsWithDataHigherThanAverage(CustomOperatorBase):
    def run(self):
        field = self.get_input(0, dpf.Field)
        average = common.compute_average_of_field(field)
        ids_in = field.scoping.ids
        data_in = field.data
        out = []
        for i, d in enumerate(data_in):
            if d >= average:
                out.append(ids_in[i])
        scoping_out = dpf.Scoping(ids=out, location=field.scoping.location)
        self.set_output(0, scoping_out)
        self.set_succeeded()

    @property
    def specification(self):
        spec = CustomSpecification("Creates a scoping with all the id

## Load Plugin
Once a python plugin is written as a package, it can be loaded with the function
:py:func:`ansys.dpf.core.core.load_library` taking as first argument the path to the directory of the plugin,
as second argument ``py_`` + any name identifying the plugin,
and as last argument the function's name exposed in the __init__ file and used to record operators.



In [4]:
import os
from ansys.dpf import core as dpf
from ansys.dpf.core import examples

tmp = dpf.make_tmp_dir_server()
dpf.upload_files_in_folder(
    dpf.path_utilities.join(tmp, "average_filter_plugin"),
    os.path.join(os.getcwd(),"../plugins", "average_filter_plugin")
)
dpf.load_library(
    os.path.join(dpf.path_utilities.join(tmp, "average_filter_plugin")),
    "py_average_filter",
    "load_operators")

Established connection to DPF gRPC


'py_average_filter successfully loaded'

Once the Plugin loaded, Operators recorded in the plugin can be used with:



In [5]:
new_operator = dpf.Operator("ids_with_data_lower_than_average")

To use this new Operator, a workflow computing the norm of the displacement
is connected to the "ids_with_data_lower_than_average" Operator.
Methods of the class ``ids_with_data_lower_than_average`` are dynamically added thanks to the Operator's
specification.



## Use the Custom Operator



In [6]:
ds = dpf.DataSources(dpf.upload_file_in_tmp_folder(examples.static_rst))
displacement = dpf.operators.result.displacement(data_sources=ds)
norm = dpf.operators.math.norm(displacement)
new_operator.inputs.connect(norm)


new_scoping = new_operator.outputs.scoping()
print("scoping in was:", norm.outputs.field().scoping)
print("----------------------------------------------")
print("scoping out is:", new_scoping)

scoping in was: DPF  Scoping: 
  with Nodal location and 81 entities

----------------------------------------------
scoping out is: DPF  Scoping: 
  with Nodal location and 35 entities

